#Imports

In [ ]:
#Sickit learn met régulièrement à jour des versions et
#indique des futurs warnings.
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
# librairies générales
import pandas as pd
import re
from tabulate import tabulate
import time
import numpy as np
import pickle
import string
import base64
import sys
# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns
# librairies scikit learn
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
# librairies des classifiers utilisés
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')
my_local_drive='/content/gdrive/My Drive/Colab Notebooks/TER'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive
%pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/TER


'/content/gdrive/My Drive/Colab Notebooks/TER'

#Classification

In [ ]:
ALEX = pd.read_excel("ALEX.xlsx")
ALEX.head()

,Unnamed: 0,Chip_Code,Chip_Type,Age,Gender,French_Residence_Department,French_Region,Rural_or_urban_area,Allergy_Present,Severe_Allergy,...,Min_Aliments Animaux,Moyenne_Moisissures,Max_Moisissures,Min_Moisissures,Moyenne_Divers,Max_Divers,Min_Divers,Moyenne_Autres,Max_Autres,Min_Autres
0,PMP0237,02AGT832,ALEX,28,F,999,7,9,0,0,...,0.0,0.060000,0.67,0,0.021667,0.12,0,1.032,4.49,0.0
1,PMP0238,02AGT834,ALEX,20,M,999,7,9,0,0,...,0.0,0.671538,5.83,0,0.576667,5.53,0,1.896,8.64,0.0
2,PMP0239,02AGT835,ALEX,22,F,999,7,9,0,0,...,0.0,1.490000,19.01,0,0.121667,0.92,0,0.426,1.66,0.0
3,PMP0240,02AGT486,ALEX,10,M,999,7,9,0,0,...,0.0,0.055385,0.22,0,0.043333,0.16,0,6.420,31.74,0.0
4,PMP0241,02AGT488,ALEX,2,M,999,7,9,0,0,...,0.0,0.026923,0.21,0,0.034167,0.20,0,0.002,0.01,0.0


In [13]:
target_1 = [
    "Allergy_Present",
    "Respiratory_Allergy",
    "Food_Allergy",
    "Venom_Allergy",
    "Severe_Allergy",
    "Type_of_Food_Allergy_Other",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA",
    "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram",
    "Type_of_Respiratory_Allergy_GINA",
    "Type_of_Food_Allergy_Aromatics",
    "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg",
    "Type_of_Food_Allergy_Fish",
    "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk",
    "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes",
    "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish",
    "Type_of_Food_Allergy_TPO",
    "Type_of_Food_Allergy_Tree_Nuts",
    "Type_of_Venom_Allergy_ATCD_Venom",
    "Type_of_Venom_Allergy_IGE_Venom",
]

extra_columns = [
    "Chip_Type",
    "Chip_Code",
    "French_Region",
    "French_Residence_Department",
    "Gender"
    ]

extra = ['History_of_food_anaphylaxis','First_degree_family_history_of_atopy', 'History_of_atopic_dematitis', 'History_of_mast_cell_disorders',
         'History_of_hymenoptera_venom_anaphylaxis','Mammalian_meat']
extra_1 = ["Conjunctivitis", "Oral_Syndrom", "Cardiovascular_symptoms", "Respiratory_symptoms", "Gastrointestinal_symptoms", "Food_polyallergies"]

Gina = ["GINA_(asthma)_0", "GINA_(asthma)_1", "GINA_(asthma)_2", "GINA_(asthma)_3", "GINA_(asthma)_4", "GINA_(asthma)_5", "GINA_(asthma)_9"]
inconnu = ["Treatment_of_athsma_9", "Treatment_of_rhinitis_9", "General_cofactors_9",
           "Age_of_onsets_9", "ARIA_(rhinitis)_9", "GINA_(asthma)_9", "Treatment_of_atopic_dematitis_9"]
Aria = ["ARIA_(rhinitis)_9", "ARIA_(rhinitis)_0", "ARIA_(rhinitis)_1", "ARIA_(rhinitis)_2", "ARIA_(rhinitis)_3", "ARIA_(rhinitis)_4", "ARIA_(rhinitis)_5"]

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

targets = ["Allergy_Present", "Respiratory_Allergy", "Food_Allergy", "Venom_Allergy"]

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

X = ALEX.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]

results = []
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for target in targets:
    y = ALEX[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"\n🔍 Target: {target} | Model: {model_name}")
        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results).to_csv("results_ALEX_Allergie.csv", index=False)


🔍 Target: Allergy_Present | Model: RandomForest
📈 Accuracy: 0.9710
🎯 F1 (0): 0.9701 | F1 (1): 0.9718
📊 Precision: 0.9717 | AUC: 0.9954517180656864
📊 Confusion Matrix:
 [[554   0]
 [  0 585]]



🔍 Target: Allergy_Present | Model: XGBoost
📈 Accuracy: 0.9605
🎯 F1 (0): 0.9595 | F1 (1): 0.9614
📊 Precision: 0.9611 | AUC: 0.990096216612143
📊 Confusion Matrix:
 [[554   0]
 [  0 585]]



🔍 Target: Allergy_Present | Model: LogisticRegression
📈 Accuracy: 0.9464
🎯 F1 (0): 0.9461 | F1 (1): 0.9467
📊 Precision: 0.9485 | AUC: 0.981369367044411
📊 Confusion Matrix:
 [[544  10]
 [ 33 552]]



🔍 Target: Allergy_Present | Model: SVM
📈 Accuracy: 0.8709
🎯 F1 (0): 0.8595 | F1 (1): 0.8804
📊 Precision: 0.8753 | AUC: 0.9319373021776587
📊 Confusion Matrix:
 [[453 101]
 [ 39 546]]



🔍 Target: Respiratory_Allergy | Model: RandomForest
📈 Accuracy: 0.9517
🎯 F1 (0): 0.9568 | F1 (1): 0.9451
📊 Precision: 0.9530 | AUC: 0.9890913952119309
📊 Confusion Matrix:
 [[644   0]
 [  0 495]]


...


🔍 Target: Food_Allergy | Model: SVM
📈 Accuracy: 0.8420
🎯 F1 (0): 0.8804 | F1 (1): 0.7659
📊 Precision: 0.8436 | AUC: 0.9047985534010465
📊 Confusion Matrix:
 [[726  27]
 [139 247]]



🔍 Target: Venom_Allergy | Model: RandomForest
📈 Accuracy: 0.9105
🎯 F1 (0): 0.9518 | F1 (1): 0.3513
📊 Precision: 0.9023 | AUC: 0.8839550264550265
📊 Confusion Matrix:
 [[1050    0]
 [   0   89]]



🔍 Target: Venom_Allergy | Model: XGBoost
📈 Accuracy: 0.9061
🎯 F1 (0): 0.9491 | F1 (1): 0.3829
📊 Precision: 0.9049 | AUC: 0.8575264550264551
📊 Confusion Matrix:
 [[1050    0]
 [   0   89]]



🔍 Target: Venom_Allergy | Model: LogisticRegression
📈 Accuracy: 0.8516
🎯 F1 (0): 0.9174 | F1 (1): 0.2575
📊 Precision: 0.8837 | AUC: 0.6994444444444443
📊 Confusion Matrix:
 [[1043    7]
 [  77   12]]



🔍 Target: Venom_Allergy | Model: SVM
📈 Accuracy: 0.7199
🎯 F1 (0): 0.8274 | F1 (1): 0.2312
📊 Precision: 0.8860 | AUC: 0.7180026455026456
📊 Confusion Matrix:
 [[1050    0]
 [  89    0]]


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

ALEX_sev = ALEX[ALEX["Allergy_Present"] == 1]
targets = ["Severe_Allergy"]
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}
X = ALEX_sev.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]
results_ALEX = []
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for target in targets:
    y = ALEX_sev[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"\n🔍 Target: {target} | Model: {model_name}")
        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results_ALEX.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results_ALEX).to_csv("results_ALEX_severe.csv", index=False)


🔍 Target: Severe_Allergy | Model: RandomForest
📈 Accuracy: 0.8360
🎯 F1 (0): 0.6370 | F1 (1): 0.8936
📊 Precision: 0.8368 | AUC: 0.8860155122655122
📊 Confusion Matrix:
 [[140   0]
 [  0 445]]



🔍 Target: Severe_Allergy | Model: XGBoost
📈 Accuracy: 0.8189
🎯 F1 (0): 0.6215 | F1 (1): 0.8804
📊 Precision: 0.8217 | AUC: 0.884549062049062
📊 Confusion Matrix:
 [[140   0]
 [  0 445]]



🔍 Target: Severe_Allergy | Model: LogisticRegression
📈 Accuracy: 0.8070
🎯 F1 (0): 0.6452 | F1 (1): 0.8667
📊 Precision: 0.8319 | AUC: 0.8610173160173161
📊 Confusion Matrix:
 [[ 96  44]
 [ 27 418]]



🔍 Target: Severe_Allergy | Model: SVM
📈 Accuracy: 0.6289
🎯 F1 (0): 0.4227 | F1 (1): 0.7232
📊 Precision: 0.7136 | AUC: 0.6493975468975469
📊 Confusion Matrix:
 [[  0 140]
 [  0 445]]


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

# Données respiratoires
ALEX_res = ALEX[ALEX["Respiratory_Allergy"] == 1]

targets = ["Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA",
    "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram",
    "Type_of_Respiratory_Allergy_GINA"]

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

X = ALEX_res.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]

results_ALEX_res = []
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Boucle principale
for target in targets:
    y = ALEX_res[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        print(f"\n🔍 Target: {target} | Model: {model_name}")

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            # Application de SMOTE sur les données d'entraînement
            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        # Entraînement final sur tout X (sans SMOTE ici, car prédiction globale)
        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results_ALEX_res.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results_ALEX_res).to_csv("results_ALEX_respiratoire.csv", index=False)


🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb | Model: RandomForest
📈 Accuracy: 0.7533
🎯 F1 (0): 0.7527 | F1 (1): 0.7520
📊 Precision: 0.7565 | AUC: 0.8329328205128206
📊 Confusion Matrix:
 [[252   0]
 [  0 243]]



🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb | Model: XGBoost
📈 Accuracy: 0.7571
🎯 F1 (0): 0.7542 | F1 (1): 0.7583
📊 Precision: 0.7619 | AUC: 0.823191282051282
📊 Confusion Matrix:
 [[252   0]
 [  0 243]]



🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb | Model: LogisticRegression
📈 Accuracy: 0.7009
🎯 F1 (0): 0.7151 | F1 (1): 0.6838
📊 Precision: 0.7043 | AUC: 0.7811423076923077
📊 Confusion Matrix:
 [[210  42]
 [ 60 183]]



🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb | Model: SVM
📈 Accuracy: 0.6991
🎯 F1 (0): 0.7099 | F1 (1): 0.6856
📊 Precision: 0.7028 | AUC: 0.802943846153846
📊 Confusion Matrix:
 [[200  52]
 [ 71 172]]



🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Tree | Model: RandomForest
📈 Accuracy: 0.8687
🎯 F1 (0): 0.8227 | F1 (1): 0.8951
📊 Precision: 0.8725 | AUC: 0.9461661125259384
📊 Confusion Matrix:
 [[182   0]
 [  0 313]]


...


🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Gram | Model: SVM
📈 Accuracy: 0.8181
🎯 F1 (0): 0.7982 | F1 (1): 0.8336
📊 Precision: 0.8564 | AUC: 0.9313657800415015
📊 Confusion Matrix:
 [[162  25]
 [ 41 267]]



🔍 Target: Type_of_Respiratory_Allergy_GINA | Model: RandomForest
📈 Accuracy: 0.9130
🎯 F1 (0): 0.9261 | F1 (1): 0.8939
📊 Precision: 0.9204 | AUC: 0.9726219581211092
📊 Confusion Matrix:
 [[304   0]
 [  0 191]]



🔍 Target: Type_of_Respiratory_Allergy_GINA | Model: XGBoost
📈 Accuracy: 0.9231
🎯 F1 (0): 0.9347 | F1 (1): 0.9062
📊 Precision: 0.9289 | AUC: 0.9716921335597059
📊 Confusion Matrix:
 [[304   0]
 [  0 191]]



🔍 Target: Type_of_Respiratory_Allergy_GINA | Model: LogisticRegression
📈 Accuracy: 0.9192
🎯 F1 (0): 0.9328 | F1 (1): 0.8983
📊 Precision: 0.9234 | AUC: 0.9801530843237126
📊 Confusion Matrix:
 [[304   0]
 [  2 189]]



🔍 Target: Type_of_Respiratory_Allergy_GINA | Model: SVM
📈 Accuracy: 0.6202
🎯 F1 (0): 0.6776 | F1 (1): 0.5313
📊 Precision: 0.6309 | AUC: 0.6561621392190153
📊 Confusion Matrix:
 [[298   6]
 [143  48]]


In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

ALEX_food = ALEX[ALEX["Food_Allergy"] == 1]
targets = ["Type_of_Food_Allergy_Aromatics",
    "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg",
    "Type_of_Food_Allergy_Fish",
    "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk",
    "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes",
    "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish",
    "Type_of_Food_Allergy_TPO",
    "Type_of_Food_Allergy_Tree_Nuts"]

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

X=ALEX_food.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]
results_food = []

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for target in targets:
    y = ALEX_food[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"\n🔍 Target: {target} | Model: {model_name}")
        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results_food.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results_food).to_csv("results_ALEX_food.csv", index=False)


🔍 Target: Type_of_Food_Allergy_Aromatics | Model: RandomForest
📈 Accuracy: 0.9251
🎯 F1 (0): 0.9608 | F1 (1): 0.0400
📊 Precision: 0.8756 | AUC: 0.744212962962963
📊 Confusion Matrix:
 [[360   0]
 [  0  26]]



🔍 Target: Type_of_Food_Allergy_Aromatics | Model: XGBoost
📈 Accuracy: 0.9017
🎯 F1 (0): 0.9474 | F1 (1): 0.1733
📊 Precision: 0.8936 | AUC: 0.7592592592592593
📊 Confusion Matrix:
 [[360   0]
 [  0  26]]



🔍 Target: Type_of_Food_Allergy_Aromatics | Model: LogisticRegression
📈 Accuracy: 0.8707
🎯 F1 (0): 0.9296 | F1 (1): 0.1436
📊 Precision: 0.8857 | AUC: 0.7171296296296296
📊 Confusion Matrix:
 [[359   1]
 [ 20   6]]



🔍 Target: Type_of_Food_Allergy_Aromatics | Model: SVM
📈 Accuracy: 0.7123
🎯 F1 (0): 0.8261 | F1 (1): 0.1400
📊 Precision: 0.8872 | AUC: 0.6351851851851852
📊 Confusion Matrix:
 [[360   0]
 [ 26   0]]



🔍 Target: Type_of_Food_Allergy_Cereals_&_Seeds | Model: RandomForest
📈 Accuracy: 0.9431
🎯 F1 (0): 0.9707 | F1 (1): 0.0000
📊 Precision: 0.9137 | AUC: 0.43384009009009006
📊 Confusion Matrix:
 [[369   0]
 [  0  17]]


...


🔍 Target: Type_of_Food_Allergy_TPO | Model: SVM
📈 Accuracy: 0.7875
🎯 F1 (0): 0.8780 | F1 (1): 0.1406
📊 Precision: 0.9036 | AUC: 0.6246746746746747
📊 Confusion Matrix:
 [[364   0]
 [ 22   0]]



🔍 Target: Type_of_Food_Allergy_Tree_Nuts | Model: RandomForest
📈 Accuracy: 0.7252
🎯 F1 (0): 0.8305 | F1 (1): 0.2604
📊 Precision: 0.6982 | AUC: 0.6592209450830141
📊 Confusion Matrix:
 [[292   0]
 [  0  94]]



🔍 Target: Type_of_Food_Allergy_Tree_Nuts | Model: XGBoost
📈 Accuracy: 0.7410
🎯 F1 (0): 0.8304 | F1 (1): 0.4378
📊 Precision: 0.7404 | AUC: 0.7172286079182632
📊 Confusion Matrix:
 [[292   0]
 [  0  94]]



🔍 Target: Type_of_Food_Allergy_Tree_Nuts | Model: LogisticRegression
📈 Accuracy: 0.6922
🎯 F1 (0): 0.7935 | F1 (1): 0.3754
📊 Precision: 0.7029 | AUC: 0.6821072796934866
📊 Confusion Matrix:
 [[279  13]
 [ 55  39]]



🔍 Target: Type_of_Food_Allergy_Tree_Nuts | Model: SVM
📈 Accuracy: 0.6213
🎯 F1 (0): 0.7105 | F1 (1): 0.4414
📊 Precision: 0.7190 | AUC: 0.6295913154533844
📊 Confusion Matrix:
 [[292   0]
 [ 93   1]]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

ALEX_venom = ALEX[ALEX["Venom_Allergy"] == 1]
targets = ["Type_of_Venom_Allergy_ATCD_Venom",
    "Type_of_Venom_Allergy_IGE_Venom"]

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

X=ALEX_venom.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]
results_venom = []

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for target in targets:
    y = ALEX_venom[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"\n🔍 Target: {target} | Model: {model_name}")
        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results_venom.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results_venom).to_csv("results_ALEX_venom.csv", index=False)

#Ne lancer pas cette partie, c pour la recherche des hyperparametres

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from xgboost import XGBClassifier

# Grille des hyperparamètres à tester
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    'gamma': [0, 1, 5]
}

# Données et targets
X = V1.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X = X.iloc[:, 1:]

targets = [
    "Allergy_Present",
    "Respiratory_Allergy",
    "Food_Allergy",
    "Venom_Allergy",
    "Severe_Allergy",
    "Type_of_Food_Allergy_Other",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA",
    "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram",
    "Type_of_Respiratory_Allergy_GINA",
    "Type_of_Food_Allergy_Aromatics",
    "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg",
    "Type_of_Food_Allergy_Fish",
    "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk",
    "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes",
    "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish",
    "Type_of_Food_Allergy_TPO",
    "Type_of_Food_Allergy_Tree_Nuts",
    "Type_of_Venom_Allergy_ATCD_Venom",
    "Type_of_Venom_Allergy_IGE_Venom",
]

all_results = []

for target in targets:
    print(f"\n🔍 Recherche d'hyperparamètres pour le target: {target}")

    y = V1[target]
    filtered_X = X.copy()
    filtered_y = y.copy()

    if target in ["Severe_Allergy", "Respiratory_Allergy", "Food_Allergy", "Venom_Allergy"]:
        mask = V1["Allergy_Present"] == 1
        filtered_X = filtered_X[mask]
        filtered_y = filtered_y[mask]
    elif target.startswith("Type_of_Respiratory_Allergy_"):
        mask = V1["Respiratory_Allergy"] == 1
        filtered_X = filtered_X[mask]
        filtered_y = filtered_y[mask]
    elif target.startswith("Type_of_Food_Allergy_"):
        mask = V1["Food_Allergy"] == 1
        filtered_X = filtered_X[mask]
        filtered_y = filtered_y[mask]

    if len(filtered_y.unique()) < 2 or len(filtered_y) < 10:
        print(f" Pas assez de données pour {target}, on passe.")
        continue

    model = XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False)

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='f1',
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1,
        verbose=1
    )

    grid_search.fit(filtered_X, filtered_y)


    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    best_params['Target'] = target
    best_params['Best_F1_Score'] = best_score

    all_results.append(best_params)

df_results = pd.DataFrame(all_results)
df_results = df_results[['Target'] + [col for col in df_results.columns if col != 'Target']]  # Remettre Target en premier
df_results.to_csv("xgboost_best_hyperparameters_V1.csv", index=False)


🔍 Recherche d'hyperparamètres pour le target: Allergy_Present
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Respiratory_Allergy
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Food_Allergy
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Venom_Allergy
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Severe_Allergy
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Type_of_Food_Allergy_Other
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Type_of_Respiratory_Allergy

#TOP Features

In [18]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import plotly.graph_objects as go

targets = [
    "Allergy_Present", "Respiratory_Allergy", "Food_Allergy", "Venom_Allergy",
    "Severe_Allergy", "Type_of_Food_Allergy_Other", "Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree", "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach", "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA", "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram", "Type_of_Respiratory_Allergy_GINA",
    "Type_of_Food_Allergy_Aromatics", "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg", "Type_of_Food_Allergy_Fish", "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk", "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes", "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish", "Type_of_Food_Allergy_TPO", "Type_of_Food_Allergy_Tree_Nuts",
    "Type_of_Venom_Allergy_ATCD_Venom", "Type_of_Venom_Allergy_IGE_Venom"
]

inconnu = ["Treatment_of_athsma_9", "Treatment_of_rhinitis_9", "General_cofactors_9",
           "Age_of_onsets_9", "ARIA_(rhinitis)_9", "GINA_(asthma)_9", "Treatment_of_atopic_dematitis_9"]

X = ALEX.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]

def plot_top_features(model, X_sub, y_sub, target):
    if len(np.unique(y_sub)) < 2:
        print(f"⚠️ Target '{target}' contient une seule classe ({np.unique(y_sub)}). Skip.")
        return

    model.fit(X_sub, y_sub)
    importances = model.feature_importances_
    top_indices = np.argsort(importances)[::-1][:10]
    features = X_sub.columns[top_indices]
    scores = importances[top_indices]

    fig = go.Figure(go.Bar(
        x=scores[::-1],
        y=features[::-1],
        orientation='h',
        text=[f"{s:.3f}" for s in scores[::-1]],
        textposition='outside'
    ))
    fig.update_layout(
        title=f"Top 10 Features pour la cible '{target}' (XGBoost)",
        xaxis_title="Importance",
        yaxis_title="Feature",
        width=800, height=500
    )
    fig.show()

for target in targets:
    X_sub = X.copy()
    y_sub = ALEX[target]
    model = XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False)
    plot_top_features(model, X_sub, y_sub, target)